In [208]:
import pandas as pd
import numpy as np
import math
from decimal import Decimal

In [209]:
def vector_norm(x):
    """Compute the L2 norm of a vector."""
    return Decimal(np.dot(x, x)).sqrt()

def qr_decomposition(A):
    """
    Compute the QR decomposition of a matrix A using the Gram-Schmidt algorithm.

    Parameters:
    A (numpy.ndarray): The matrix to decompose.

    Returns:
    (numpy.ndarray, numpy.ndarray): A tuple of Q and R matrices that represent the QR decomposition of A, where:
        Q (numpy.ndarray): The orthogonal matrix Q.
        R (numpy.ndarray): The upper triangular matrix R.
    """
    # Get the shape of the input matrix
    m, n = A.shape

    # Initialize the matrices
    Q = np.zeros((m, n))
    R = np.zeros((n, n))

    # Perform the Gram-Schmidt orthogonalization
    for j in range(n):
        v = A[:, j]
        for i in range(j):
            R[i, j] = np.dot(Q[:, i], A[:, j])
            v = v - R[i, j] * Q[:, i]
        # R[j, j] = np.linalg.norm(v)
        R[j, j] = vector_norm(v)
        Q[:, j] = v / R[j, j]

    return Q, R

In [210]:
def rank_of_matrix(mat):
    """
    This function calculates the rank of a matrix 'mat' using Gaussian elimination method.
    It returns the rank of the matrix.
    """
    # Define the dimensions of the matrix
    m = len(mat)
    n = len(mat[0])

    rank = min(m, n)

    # Perform Gaussian elimination
    for row in range(rank):
        # Check if the diagonal element is not zero
        if mat[row][row] != 0:
            for col in range(row + 1, m):
                # Calculate the factor by which to multiply the current row
                # to eliminate the non-zero element in the current column
                factor = mat[col][row] / mat[row][row]
                for i in range(row, n):
                    # Update the current row by subtracting the product of the factor
                    # and the corresponding element in the row being eliminated from it
                    mat[col][i] -= factor * mat[row][i]
        else:
            # If the diagonal element is zero, look for a non-zero element below it
            # and swap the rows if necessary
            reduce_rank = True
            for i in range(row + 1, m):
                if mat[i][row] != 0:
                    mat[row], mat[i] = mat[i], mat[row]
                    reduce_rank = False
                    break
            if reduce_rank:
                rank -= 1
                for i in range(row, m):
                    mat[i][row] = mat[i][rank]

    return rank


In [211]:
def eig(A):
    """
    Compute the eigenvalues and eigenvectors of a matrix A using the power iteration method.

    Parameters:
    A (numpy.ndarray): The matrix to compute eigenvalues and eigenvectors.

    Returns:
    (eigvals, eigvecs): A tuple of arrays that represent the eigenvalues and eigenvectors of A, where:
        eigvals (numpy.ndarray): The eigenvalues of A.
        eigvecs (numpy.ndarray): The eigenvectors of A.
    """
    # set the number of iterations and tolerance level
    max_iter = 100
    tol = 1e-6

    # initialize the eigenvectors
    m, n = A.shape
    eigvecs = np.random.randn(n, n)

    # compute the largest eigenvalue and eigenvector
    for i in range(max_iter):
        # compute the new eigenvector
        eigvecs_new = A @ eigvecs
        # eigvecs_new, _ = np.linalg.qr(eigvecs_new)
        eigvecs_new, _ = qr_decomposition(eigvecs_new)
        if np.allclose(eigvecs_new, eigvecs, rtol=tol):
            break
        eigvecs = eigvecs_new

    # compute the eigenvalues
    eigvals = np.diag(eigvecs.T @ A @ eigvecs)

    return eigvals, eigvecs


In [212]:
# def SVD(A):
#     # Compute the eigenvectors and eigenvalues of A*At or At*A, whichever is smaller
#     if A.shape[0] < A.shape[1]:
#         k=np.linalg.matrix_rank(A@A.T)
#         S = np.dot(A, A.T)
#     else:
#         k=np.linalg.matrix_rank(A.T@A)
#         S = np.dot(A.T, A)
        
#     # eigvals, eigvecs = np.linalg.eig(S) #NOT ALLOWED
#     sigma, eigenvectors = eig(S)
#     eigenvectors =eigenvectors[:k]
#     # Compute the singular values and their reciprocals
#     sigma=np.diag(np.sqrt(sigma[:k]))

#     # Compute the left and right singular vectors
#     if(A.shape[0] < A.shape[1]):
#         V = A@eigenvectors@np.linalg.inv(sigma)@eigenvectors.T
#         U = eigenvectors
#     else:
#         U = A@eigenvectors@np.linalg.inv(sigma)
#         V=eigenvectors

#     return U,sigma, V.T

In [213]:
def SVD(A):
    """
    Compute Singular Value Decomposition of matrix A using NumPy.

    Args:
        A: numpy.array, matrix to be decomposed

    Returns:
        U: numpy.array, matrix containing left singular vectors
        s: numpy.array, array containing singular values
        V_T: numpy.array, matrix containing right singular vectors (transposed)
    """
    # Compute the eigenvectors and eigenvalues of A*At or At*A, whichever is smaller
    if A.shape[0] < A.shape[1]:
        # S = np.dot(A, A.T)
        S = A @ A.T
        # print(A @ A.T)
        # k = np.linalg.matrix_rank(S)
        k = rank_of_matrix(S.copy())
    else:
        # S = np.dot(A.T, A)
        S = A.T @ A
        # k = np.linalg.matrix_rank(S)
        k = rank_of_matrix(S.copy())
        
    # eigvals, eigvecs = np.linalg.eig(S) #NOT ALLOWED
    print(S)
    eigvals, eigvecs = eig(S)
    print(eigvals)
    # Sort the eigenvectors by descending eigenvalues
    sorted_indices = np.argsort(eigvals)[::-1]
    eigvals = eigvals[sorted_indices]
    eigvecs = eigvecs[:,sorted_indices]

    # Compute the singular values and their reciprocals
    s = np.sqrt(eigvals)
    # s = s[s > 10e-6]
    s = s[:k]
    s_inv = np.zeros_like(A.T)
    np.fill_diagonal(s_inv, 1.0 / s)

    # Compute the left and right singular vectors
    # if(A.shape[0] > A.shape[1]):
    if(A.shape[0] > A.shape[1]):
        U = np.dot(A, np.dot(eigvecs, s_inv))
        V_T = eigvecs.T
        if(len(s) != V_T.shape[0]): V_T = V_T[:len(s) - V_T.shape[0], :] 

    else:
        U = eigvecs
        V_T = np.dot(s_inv, np.dot(U.T, A))
        if(len(s) != U.shape[1]): U = U[:, :len(s) - U.shape[1]]


    
    # else: 
    #     V_T = np.dot(s_inv, np.dot(eigvecs, A))
    #     U = eigvecs.T
    #     if(len(s) != V_T.shape[0]): V_T = V_T[:len(s) - V_T.shape[0], :]

    sigma = np.zeros([U.shape[1], V_T.shape[0]])
    # sigma = np.zeros([len(s), len(s)])
    # sigma = np.diag(s)
    # if(A.shape[0] < A.shape[1]):
    for i in range(len(s)):
        sigma[i, i] = s[i]
    # else:
    #     for i in range(len(s)):
    #         sigma[i, i] = s[i]

    return U, s, sigma, V_T

In [214]:
# tryy = np.array([[0.89411765, 0.84117647, 0.77058824, 0.54411765, 0.91176471,
#         0.784375  , 0.7625    , 0.61785714, 0.69444444, 0.76666667,
#         0.6875    , 0.58181818, 0.62222222, 0.44      , 0.6       ,
#         0.48333333, 0.45      , 0.41666667, 0.48333333, 0.5       ,
#         0.28571429, 0.47166667, 0.52166667, 0.45      , 0.72142857],
#        [0.75543947, 0.63583333, 0.56670589, 0.52787547, 0.82429796,
#         0.86477368, 0.69059243, 0.64201142, 0.6202247 , 0.63628521,
#         0.65243902, 0.45059658, 0.59061035, 0.54418605, 0.61658815,
#         0.23793103, 0.24285714, 0.15517241, 0.21785714, 0.12962963,
#         0.33333333, 0.50517084, 0.43846154, 0.34615385, 0.57777778],
#        [0.81939394, 0.67545455, 0.65515152, 0.65939394, 0.83606061,
#         0.803     , 0.809     , 0.61724138, 0.5875    , 0.675     ,
#         0.66666667, 0.42884615, 0.66086957, 0.6       , 0.55      ,
#         0.19047619, 0.2       , 0.225     , 0.195     , 0.15789474,
#         0.31666667, 0.4568    , 0.455     , 0.5       , 0.45714286],
#        [0.77526316, 0.69210526, 0.61526316, 0.72684211, 0.9       ,
#         0.67692308, 0.58571429, 0.54545455, 0.53333333, 0.52222222,
#         0.75555556, 0.30111111, 0.47777778, 0.56666667, 0.75555556,
#         0.14571429, 0.11714286, 0.04857143, 0.12142857, 0.03333333,
#         0.33333333, 0.38      , 0.35      , 0.45      , 0.52857143]])

In [215]:
tryy = np.array([[0.76840567, 0.67184615, 0.61557467, 0.55142351, 0.86119812,
        0.74003927, 0.67927052, 0.56742435, 0.56428521, 0.60080851,
        0.59384615, 0.47652097, 0.59850127, 0.58923077, 0.58251578,
        0.43261538, 0.40184615, 0.35753846, 0.40538462, 0.36      ,
        0.41615385, 0.49341019, 0.45846154, 0.46153846, 0.52692308],
       [0.86451613, 0.79419355, 0.72419355, 0.66451613, 0.87806452,
        0.81129032, 0.73709677, 0.65032258, 0.59322581, 0.6383871 ,
        0.53387097, 0.50806452, 0.56612903, 0.48064516, 0.49677419,
        0.39354839, 0.41935484, 0.41612903, 0.42580645, 0.43225806,
        0.45      , 0.41483871, 0.46548387, 0.44677419, 0.52258065],
       [0.81538462, 0.71153846, 0.48461538, 0.65384615, 0.87692308,
        0.75      , 0.70769231, 0.46923077, 0.58461538, 0.52307692,
        0.5       , 0.42384615, 0.40769231, 0.36153846, 0.44615385,
        0.42307692, 0.46153846, 0.46153846, 0.43846154, 0.34615385,
        0.46153846, 0.46923077, 0.43846154, 0.5       , 0.46923077],
       [0.775     , 0.56388889, 0.46111111, 0.61111111, 0.82777778,
        0.82777778, 0.675     , 0.65      , 0.44166667, 0.46111111,
        0.71111111, 0.53055556, 0.56111111, 0.52777778, 0.55555556,
        0.31666667, 0.33333333, 0.32777778, 0.30555556, 0.32777778,
        0.41111111, 0.59166667, 0.55555556, 0.52777778, 0.65      ],
       [0.85      , 0.66111111, 0.71944444, 0.53611111, 0.84722222,
        0.78333333, 0.68333333, 0.61944444, 0.49444444, 0.48888889,
        0.73333333, 0.4       , 0.51111111, 0.58333333, 0.60555556,
        0.27777778, 0.27777778, 0.32222222, 0.26666667, 0.27777778,
        0.39722222, 0.41666667, 0.50555556, 0.45555556, 0.38333333]])

In [216]:
U, s, V_trans = np.linalg.svd(tryy)
print(s)


[6.2692342  0.47502054 0.36730407 0.23289804 0.1730365 ]


In [217]:
U1, s1, sigma, V_trans1 = SVD(tryy)
s1

[[7.98925634 8.32793366 7.61913971 7.85361259 7.73200037]
 [8.32793366 8.77652353 8.01341688 8.16654034 8.07810572]
 [7.61913971 8.01341688 7.43973432 7.49926497 7.35349144]
 [7.85361259 8.16654034 7.49926497 7.89853398 7.65212963]
 [7.73200037 8.07810572 7.35349144 7.65212963 7.64398917]]
[3.93032974e+01 2.25644514e-01 1.34912278e-01 5.42414992e-02
 2.99416312e-02]


array([6.2692342 , 0.47502054, 0.36730407, 0.23289804, 0.1730365 ])

In [218]:
tryy @ tryy.T

array([[7.98925634, 8.32793366, 7.61913971, 7.85361259, 7.73200037],
       [8.32793366, 8.77652353, 8.01341688, 8.16654034, 8.07810572],
       [7.61913971, 8.01341688, 7.43973432, 7.49926497, 7.35349144],
       [7.85361259, 8.16654034, 7.49926497, 7.89853398, 7.65212963],
       [7.73200037, 8.07810572, 7.35349144, 7.65212963, 7.64398917]])

In [219]:
v1, e1 = np.linalg.eig(tryy @ tryy.T)
v1

array([3.93032974e+01, 2.99416312e-02, 5.42414992e-02, 1.34912278e-01,
       2.25644514e-01])

In [220]:
v2, e2 = eig(tryy @ tryy.T)
v2

array([3.93032974e+01, 2.25644514e-01, 1.34912278e-01, 5.42414992e-02,
       2.99416312e-02])

In [221]:
v2 = np.sqrt(v2)
v2

array([6.2692342 , 0.47502054, 0.36730407, 0.23289804, 0.1730365 ])

In [222]:
k = rank_of_matrix(tryy @ tryy.T)


In [223]:
v2[:k]

array([6.2692342 , 0.47502054, 0.36730407, 0.23289804, 0.1730365 ])